In [1]:
# dataset download path = 'https://www.microsoft.com/en-us/download/details.aspx?id=54765&WT.mc_id=rss_alldownloads_devresources'
import os
from os.path import dirname, join
from os import listdir
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import pydot
from matplotlib import pyplot as plt
from keras import Input, Model, layers, Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Conv1D, MaxPool1D, Flatten
import cv2
import time
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16

cur_dir = os.getcwd()
dataset_dir = join(cur_dir, '../dataset_cats_dogs/PetImages')
cats_dir = join(dataset_dir, 'Cat')
dogs_dir = join(dataset_dir, 'Dog')



Using TensorFlow backend.


In [2]:
Model = VGG16()
Model.summary()

W0925 14:54:17.939819 139772183480128 deprecation_wrapper.py:119] From /home/nandagopal/PycharmProjects/ImageClassifierApp/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0925 14:54:17.958698 139772183480128 deprecation_wrapper.py:119] From /home/nandagopal/PycharmProjects/ImageClassifierApp/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0925 14:54:17.962875 139772183480128 deprecation_wrapper.py:119] From /home/nandagopal/PycharmProjects/ImageClassifierApp/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0925 14:54:18.015339 139772183480128 deprecation_wrapper.py:119] From /home/nandagopal/PycharmProjects/ImageClassifierApp/venv/lib/python3.6/site

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
type(Model)

keras.engine.training.Model

In [4]:
model = Sequential()

for layer in Model.layers[:-1]:
    model.add(layer)

In [5]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [6]:
for layer in model.layers:
    layer.trainable = False

In [7]:
model.add(Dense(2, activation='softmax'))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [9]:

final_data = []
Class = []
dirs = {cats_dir : 0, dogs_dir: 1}

for folder, class_value in dirs.items():
    for file in listdir(folder)[:100]:
        if class_value == 0 :
            c = np.array([0, 1])
        else:
            c = np.array([1, 0])
        try :
            file_path = join(folder, file)
            image_array_grey_scale = cv2.resize(cv2.imread(file_path), dsize=(224, 224))   
#             reshaped = image_array_grey_scale.reshape(14400, )
            final_data.append(image_array_grey_scale)
            Class.append(c)
        except :
            print("Failed ---->", file)

Failed ----> 5553.jpg


In [10]:
final_data = np.array(final_data)
Class = np.array(Class)


print(final_data.shape)
print(Class.shape)

(199, 224, 224, 3)
(199, 2)


In [11]:
model.compile(optimizer='Adam', loss ='binary_crossentropy', metrics=['accuracy'])

W0925 14:54:22.132870 139772183480128 deprecation_wrapper.py:119] From /home/nandagopal/PycharmProjects/ImageClassifierApp/venv/lib/python3.6/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0925 14:54:22.189480 139772183480128 deprecation.py:323] From /home/nandagopal/PycharmProjects/ImageClassifierApp/venv/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
model.fit(final_data, Class, epochs=5)

Epoch 1/5
199/199 [==============================] - 26s 130ms/step - loss: 0.5043 - acc: 0.7940
Epoch 2/5
199/199 [==============================] - 26s 133ms/step - loss: 0.0669 - acc: 0.9698
Epoch 3/5
199/199 [==============================] - 27s 133ms/step - loss: 0.0253 - acc: 0.9899
Epoch 4/5
199/199 [==============================] - 26s 130ms/step - loss: 0.0074 - acc: 1.0000
Epoch 5/5
199/199 [==============================] - 27s 134ms/step - loss: 0.0012 - acc: 1.0000


In [13]:
model.predict_proba(final_data[-2:-1])

array([[9.9999774e-01, 2.2822817e-06]], dtype=float32)

In [14]:
model.save('trained_vgg16.h5')

In [15]:
from keras.models import load_model

In [16]:
load_model('trained_vgg16.h5')

ValueError: You are trying to load a weight file containing 16 layers into a model with 0 layers